In [7]:
from dash import Dash, dash_table, dcc, html, Output, Input
import pandas as pd
import plotly.graph_objs as go
import requests

In [8]:
# Step 1: Fetch data from DeFiLlama API
def fetch_data():
    url = 'https://api.llama.fi/v2/chains'
    response = requests.get(url)
    if response.status_code == 200: # data was fetched successfully
        data = response.json()
        return pd.DataFrame(data)
    else:
        raise Exception('Error fetching data from DeFiLlama API')

def fetch_historical_data(chain_name, metric):
    if metric == 'tvl':
        url = f'https://api.llama.fi/v2/historicalChainTvl/{chain_name}'
    elif metric == 'volume':
        url = (f'https://api.llama.fi/overview/options/{chain_name}'
                  '?excludeTotalDataChart=true'
                  '&excludeTotalDataChartBreakdown=true'
                  '&dataType=dailyPremiumVolume')
    elif metric == 'fee':
        url = (f'https://api.llama.fi/overview/fees/{chain_name}'
                  '?excludeTotalDataChart=true'
                  '&excludeTotalDataChartBreakdown=true'
                  '&dataType=dailyFees')
    else:
        raise ValueError('Invalid metric specified')
    
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data)  # Directly create DataFrame from the list of dictionaries
        df['date'] = pd.to_datetime(df['date'], unit='s')
        print(df.tail(120))
        return df[['date', metric]].tail(120)  # Get the last 120 days of the specified metric
    else:
        raise Exception(f'Error fetching historical {metric} data for {chain_name}')
    

# Step 2: Prepare the data
def prepare_data(df):
    # Keep only name and tvl
    df = df[['name', 'tvl']]
    # Sort the data to get the top 10 chains by TVL
    df = df.sort_values(by='tvl', ascending=False).head(10)
    return df



In [9]:
# Step 3: Create the Dash app and the layout
def create_dashboard(df):
    app = Dash(__name__)
    app.layout = html.Div([
        html.H1('DeFi Chains Dashboard'),
        dcc.RadioItems(
            options=[
                {'label': 'Total Value Locked (TVL)', 'value': 'tvl'},
                {'label': 'Volume', 'value': 'volume'},
                {'label': 'Fees', 'value': 'fee'}
            ],
            id='metric-dropdown',
            value='tvl'
        ),
        dcc.Graph(id='historical-chart'),
        dash_table.DataTable(
            data=df.to_dict('records'),
            page_size=10,
            style_table={'overflowX': 'auto'}
        )
    ])

    # Step 4: Add callback to update the chart based on selected metric
    @app.callback(
        Output(component_id='historical-chart', component_property='figure'),
        Input(component_id='metric-dropdown', component_property='value')
    )
    def update_charts(selected_metric):
        # Line Chart for historical data of top 10 chains
        historical_fig = go.Figure()
        for chain_name in df['name']:
            historical_data_df = fetch_historical_data(chain_name, selected_metric)
            historical_fig.add_trace(go.Scatter(
                x=historical_data_df['date'],
                y=historical_data_df[selected_metric],
                mode='lines',
                name=chain_name
            ))
        historical_fig.update_layout(title=f'Historical {selected_metric.upper()} of Top 10 Chains (Last 120 Days)')
        
        return historical_fig

    return app

In [ ]:
# Step 5: Run the app if the script is executed directly
if __name__ == '__main__':
    df = fetch_data()
    df = prepare_data(df)
    app = create_dashboard(df)
    app.run_server(debug=True, use_reloader=False)  # Disable reloader to prevent multiple fetches of data

           date           tvl
2280 2024-07-24  6.108826e+10
2281 2024-07-25  5.661101e+10
2282 2024-07-26  5.797353e+10
2283 2024-07-27  5.864812e+10
2284 2024-07-28  5.824549e+10
...         ...           ...
2395 2024-11-16  5.936128e+10
2396 2024-11-17  5.964766e+10
2397 2024-11-18  5.889131e+10
2398 2024-11-19  5.966322e+10
2399 2024-11-20  5.962561e+10

[120 rows x 2 columns]
           date           tvl
1224 2024-07-24  5.154377e+09
1225 2024-07-25  4.928393e+09
1226 2024-07-26  5.181171e+09
1227 2024-07-27  5.363488e+09
1228 2024-07-28  5.331117e+09
...         ...           ...
1339 2024-11-16  7.638529e+09
1340 2024-11-17  8.199497e+09
1341 2024-11-18  8.366407e+09
1342 2024-11-19  8.437207e+09
1343 2024-11-20  8.270806e+09

[120 rows x 2 columns]
           date           tvl
1573 2024-07-24  8.360186e+09
1574 2024-07-25  8.191020e+09
1575 2024-07-26  8.428692e+09
1576 2024-07-27  8.535056e+09
1577 2024-07-28  8.602785e+09
...         ...           ...
1688 2024-11-16  7.909